#### blip 사용해서 캡션 만들기

In [ ]:
import os
import json
from PIL import Image
import torch
from tqdm import tqdm
from transformers import AutoProcessor, Blip2ForImageTextRetrieval

# ====== 설정 ======
image_base_dir = "/content/drive/MyDrive/컨퍼런스/스타일추천/한국인얼굴/image_celeb/image_celeb/image_cropped"
output_dir = "/content/drive/MyDrive/korean_celebrity_makeup_captions"
batch_size = 50

# 출력 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# ====== 키워드 카테고리 정의 ======
categories = {
    "분위기": [
        "꾸안꾸", "내추럴", "데일리", "세련됨", "모던", "청순", "러블리", "차분",
        "시크", "트렌디", "우아", "럭셔리", "발랄", "청량", "페미닌", "섹시",
        "심플", "밝은", "은은한", "깔끔", "고급스러운", "쿨", "매혹적",
        "귀여운", "깜찍", "사랑스러운", "활기찬", "상큼", "성숙", "클래식"
    ],
    "눈": [
        "내추럴아이", "스모키아이", "세미스모키", "글리터아이", "매트아이",
        "브라운아이", "핑크아이", "코랄아이", "오렌지아이",
        "언더라인", "또렷한라인", "자연스러운라인", "음영"
    ],
    "피부": [
        "광채피부", "물광피부", "윤기피부", "매트피부", "보송피부", "수분피부",
        "촉촉피부", "투명피부", "글로우", "하이라이트", "컨투어", "쉐딩", "톤업"
    ],
    "블러셔": [
        "핑크블러셔", "코랄블러셔", "피치블러셔", "오렌지블러셔"
    ],
    "입술": [
        "레드립", "핑크립", "코랄립", "누드립", "브라운립", "오렌지립", "로즈립",
        "매트립", "글로시립", "그라데이션립", "자연스러운립", "또렷한립라인"
    ],
    "눈썹": [
        "자연스러운눈썹", "아치눈썹", "일자눈썹", "진한눈썹", "연한눈썹"
    ],
    "속눈썹": [
        "자연스러운속눈썹", "볼륨속눈썹", "긴속눈썹", "컬링속눈썹", "볼륨마스카라", "롱래쉬마스카라"
    ],
    "톤": [
        "웜톤", "쿨톤", "뉴트럴톤", "봄웜", "여름쿨", "가을웜", "겨울쿨"
    ]
}

# 한영 번역 딕셔너리
keyword_translation = {
    # 분위기
    "꾸안꾸": "effortless style", "내추럴": "natural beauty", "데일리": "everyday look",
    "세련됨": "sophisticated style", "모던": "modern style", "청순": "innocent beauty",
    "러블리": "lovely charm", "차분": "calm elegance", "시크": "chic style",
    "트렌디": "trendy look", "우아": "elegant style", "럭셔리": "luxury look",
    "발랄": "bright energy", "청량": "fresh beauty", "페미닌": "feminine beauty",
    "섹시": "sexy appeal", "심플": "simple style", "밝은": "bright mood",
    "은은한": "subtle beauty", "깔끔": "clean look", "고급스러운": "luxurious style",
    "쿨": "cool style", "매혹적": "alluring charm", "귀여운": "cute style",
    "깜찍": "adorable look", "사랑스러운": "lovable beauty", "활기찬": "energetic look",
    "상큼": "fresh charm", "성숙": "mature elegance", "클래식": "classic style",

    # 눈
    "내추럴아이": "natural eyes", "스모키아이": "smoky eyes", "세미스모키": "semi-smoky eyes",
    "글리터아이": "glitter eyes", "매트아이": "matte eyes", "브라운아이": "brown eyeshadow",
    "핑크아이": "pink eyeshadow", "코랄아이": "coral eyeshadow", "오렌지아이": "orange eyeshadow",
    "언더라인": "lower eyeliner", "또렷한라인": "defined eyeliner", "자연스러운라인": "natural eyeliner",
    "음영": "eye shadow depth",

    # 피부
    "광채피부": "glowing skin", "물광피부": "dewy glass skin", "윤기피부": "radiant skin",
    "매트피부": "matte skin", "보송피부": "soft velvet skin", "수분피부": "hydrated skin",
    "촉촉피부": "moist skin", "투명피부": "clear skin", "글로우": "glow effect",
    "하이라이트": "highlight", "컨투어": "contour", "쉐딩": "shading", "톤업": "tone-up effect",

    # 블러셔
    "핑크블러셔": "pink blush", "코랄블러셔": "coral blush",
    "피치블러셔": "peach blush", "오렌지블러셔": "orange blush",

    # 입술
    "레드립": "red lips", "핑크립": "pink lips", "코랄립": "coral lips",
    "누드립": "nude lips", "브라운립": "brown lips", "오렌지립": "orange lips",
    "로즈립": "rose lips", "매트립": "matte lips", "글로시립": "glossy lips",
    "그라데이션립": "gradient lips", "자연스러운립": "natural lips", "또렷한립라인": "defined lip line",

    # 눈썹
    "자연스러운눈썹": "natural brows", "아치눈썹": "arched brows", "일자눈썹": "straight brows",
    "진한눈썹": "bold brows", "연한눈썹": "light brows",

    # 속눈썹
    "자연스러운속눈썹": "natural lashes", "볼륨속눈썹": "volume lashes", "긴속눈썹": "long lashes",
    "컬링속눈썹": "curled lashes", "볼륨마스카라": "volume mascara", "롱래쉬마스카라": "lengthening mascara",

    # 톤
    "웜톤": "warm tone", "쿨톤": "cool tone", "뉴트럴톤": "neutral tone",
    "봄웜": "spring warm", "여름쿨": "summer cool", "가을웜": "autumn warm", "겨울쿨": "winter cool"
}

# ====== BLIP2 모델 초기화 ======
def initialize_blip2_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🎯 사용 디바이스: {device}")

    model_name = "Salesforce/blip2-itm-vit-g"
    print(f"🤖 BLIP2 모델 로딩 중: {model_name}")

    processor = AutoProcessor.from_pretrained(model_name)
    model = Blip2ForImageTextRetrieval.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    print("✅ BLIP2 모델 로딩 완료!")
    return processor, model, device

# ====== 이미지 경로 수집 ======
def collect_all_image_paths(base_dir):
    image_paths = []

    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')) and not file.startswith('._'):
                full_path = os.path.join(root, file)
                image_paths.append(full_path)

    return image_paths

# ====== 유사도 계산 ======
def calculate_category_similarities(image, processor, model, device, categories, keyword_translation):
    category_results = {}

    for cat_name, korean_keywords in categories.items():
        english_texts = []
        valid_korean_keywords = []

        for korean_kw in korean_keywords:
            if korean_kw in keyword_translation:
                english_texts.append(keyword_translation[korean_kw])
                valid_korean_keywords.append(korean_kw)

        if not english_texts:
            continue

        try:
            inputs = processor(
                images=image,
                text=english_texts,
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                logits_per_image = outputs.logits_per_image
                probs = logits_per_image.softmax(dim=1)[0]

            best_idx = torch.argmax(probs).item()
            best_korean_keyword = valid_korean_keywords[best_idx]
            best_english_keyword = english_texts[best_idx]
            best_score = probs[best_idx].item()

            category_results[cat_name] = {
                "korean_keyword": best_korean_keyword,
                "english_keyword": best_english_keyword,
                "score": best_score
            }

        except Exception as e:
            print(f"❌ [{cat_name}] 유사도 계산 실패: {e}")
            continue

    return category_results

# ====== 캡션 생성 ======
def generate_makeup_captions(category_results):
    detail_categories = ["눈", "피부", "블러셔", "입술", "눈썹", "속눈썹"]

    detail_results = []
    for cat in detail_categories:
        if cat in category_results:
            detail_results.append({
                "category": cat,
                "korean_keyword": category_results[cat]["korean_keyword"],
                "english_keyword": category_results[cat]["english_keyword"],
                "score": category_results[cat]["score"]
            })

    detail_results.sort(key=lambda x: x["score"], reverse=True)
    top_2_details = detail_results[:2]

    captions = {}

    # 캡션 타입 1: 분위기 1개 + 메이크업 디테일 2개
    if "분위기" in category_results and len(top_2_details) >= 2:
        mood = category_results["분위기"]
        detail1 = top_2_details[0]
        detail2 = top_2_details[1]

        captions["caption_type1"] = {
            "korean": f"{mood['korean_keyword']}, {detail1['korean_keyword']}, {detail2['korean_keyword']}",
            "english": f"{mood['english_keyword']}, {detail1['english_keyword']}, {detail2['english_keyword']}",
            "components": {
                "mood": mood,
                "detail1": detail1,
                "detail2": detail2
            }
        }

    # 캡션 타입 2: 톤 1개 + 메이크업 디테일 2개
    if "톤" in category_results and len(top_2_details) >= 2:
        tone = category_results["톤"]
        detail1 = top_2_details[0]
        detail2 = top_2_details[1]

        captions["caption_type2"] = {
            "korean": f"{tone['korean_keyword']}, {detail1['korean_keyword']}, {detail2['korean_keyword']}",
            "english": f"{tone['english_keyword']}, {detail1['english_keyword']}, {detail2['english_keyword']}",
            "components": {
                "tone": tone,
                "detail1": detail1,
                "detail2": detail2
            }
        }

    return captions

# ====== 메인 실행 함수 ======
def process_all_celebrity_images():
    print("🚀 한국인 얼굴 메이크업 캡션 생성 시작!")

    # 1. BLIP2 모델 초기화
    processor, model, device = initialize_blip2_model()

    # 2. 이미지 경로 수집
    print(f"\n📸 이미지 경로 수집 중...")
    image_paths = collect_all_image_paths(image_base_dir)
    total_images = len(image_paths)

    print(f"📊 총 발견된 이미지: {total_images}개")

    # 연예인별 이미지 개수 확인
    celeb_count = {}
    for path in image_paths:
        celeb_name = os.path.basename(os.path.dirname(path))
        celeb_count[celeb_name] = celeb_count.get(celeb_name, 0) + 1

    print(f"👥 총 연예인 수: {len(celeb_count)}명")

    # 3. 전체 이미지 처리
    all_results = []

    print(f"\n💄 캡션 생성 시작 (전체 {total_images}개 이미지)")

    for i, image_path in enumerate(tqdm(image_paths, desc="캡션 생성 중")):
        try:
            # 이미지 로드
            image = Image.open(image_path).convert("RGB")

            # 카테고리별 유사도 계산
            category_results = calculate_category_similarities(
                image, processor, model, device, categories, keyword_translation
            )

            # 캡션 생성
            captions = generate_makeup_captions(category_results)

            # 결과 저장
            result = {
                "index": i,
                "image_path": image_path,
                "image_name": os.path.basename(image_path),
                "celebrity_name": os.path.basename(os.path.dirname(image_path)),
                "category_results": category_results,
                "captions": captions
            }
            all_results.append(result)

            # 배치 저장 (batch_size개마다)
            if (i + 1) % batch_size == 0:
                batch_num = (i + 1) // batch_size
                batch_save_path = os.path.join(output_dir, f"batch_{batch_num:03d}_results.json")

                # 현재 배치만 저장
                current_batch = all_results[-batch_size:]

                with open(batch_save_path, "w", encoding="utf-8") as f:
                    json.dump(current_batch, f, indent=2, ensure_ascii=False)

                print(f"\n💾 배치 {batch_num} 저장 완료 ({i + 1}/{total_images}, {((i + 1)/total_images)*100:.1f}%)")

                # GPU 메모리 정리
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

        except Exception as e:
            print(f"\n❌ 이미지 처리 실패 (인덱스 {i}, {os.path.basename(image_path)}): {e}")
            continue

    # 마지막 배치 저장 (남은 이미지들)
    if len(all_results) % batch_size != 0:
        final_batch_num = (len(all_results) // batch_size) + 1
        final_batch_path = os.path.join(output_dir, f"batch_{final_batch_num:03d}_final.json")

        remaining_results = all_results[-(len(all_results) % batch_size):]

        with open(final_batch_path, "w", encoding="utf-8") as f:
            json.dump(remaining_results, f, indent=2, ensure_ascii=False)

        print(f"\n💾 최종 배치 저장 완료: {final_batch_path}")

    # 4. 최종 통합 결과 저장
    final_save_path = os.path.join(output_dir, "all_celebrity_makeup_captions.json")
    with open(final_save_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    # 5. 통계 생성
    stats = {
        "total_processed": len(all_results),
        "successful_caption_type1": sum(1 for r in all_results if "caption_type1" in r.get("captions", {})),
        "successful_caption_type2": sum(1 for r in all_results if "caption_type2" in r.get("captions", {})),
        "celebrities": celeb_count,
        "total_categories": len(categories),
        "total_keywords": sum(len(keywords) for keywords in categories.values())
    }

    stats_save_path = os.path.join(output_dir, "processing_statistics.json")
    with open(stats_save_path, "w", encoding="utf-8") as f:
        json.dump(stats, f, indent=2, ensure_ascii=False)

    # 6. 결과 출력
    print(f"\n🎉 처리 완료!")
    print(f"📊 총 처리된 이미지: {len(all_results)}개")
    print(f"📊 캡션 타입1 성공: {stats['successful_caption_type1']}개")
    print(f"📊 캡션 타입2 성공: {stats['successful_caption_type2']}개")
    print(f"💾 최종 결과: {final_save_path}")
    print(f"📈 통계 파일: {stats_save_path}")

    # 7. 결과 미리보기
    if all_results:
        print(f"\n👀 결과 미리보기 (첫 3개):")
        for i, result in enumerate(all_results[:3]):
            print(f"\n🖼️ {i+1}. {result['image_name']} ({result['celebrity_name']})")

            if "captions" in result:
                if "caption_type1" in result["captions"]:
                    caption1 = result["captions"]["caption_type1"]
                    print(f"   📝 캡션1: {caption1['korean']}")
                    print(f"   📝 English: {caption1['english']}")

                if "caption_type2" in result["captions"]:
                    caption2 = result["captions"]["caption_type2"]
                    print(f"   📝 캡션2: {caption2['korean']}")
                    print(f"   📝 English: {caption2['english']}")

    return all_results, stats

# ====== 실행 ======
if __name__ == "__main__":
    results, statistics = process_all_celebrity_images()

### 캡션 문장으로 바꾸기

In [ ]:
import os
import json

# 경로 설정
input_dir = "/content/drive/MyDrive/korean_celebrity_makeup_captions"
output_dir = "/content/drive/MyDrive/컨퍼런스/스타일추천/한국인얼굴/korean_celebrity_makeup_sentences"
os.makedirs(output_dir, exist_ok=True)

# ====== 문장 생성 함수들 ======
def make_caption_from_components(components):
    """캡션 컴포넌트를 자연스러운 문장으로 변환"""
    keywords = []

    # 컴포넌트에서 영어 키워드 추출
    for key, value in components.items():
        if isinstance(value, dict) and 'english_keyword' in value:
            keywords.append(value['english_keyword'])

    if not keywords:
        return ""

    if len(keywords) == 1:
        return f"This makeup look features {keywords[0]}."
    elif len(keywords) == 2:
        return f"This makeup look features {keywords[0]} and {keywords[1]}."
    else:
        return f"This makeup look features {', '.join(keywords[:-1])}, and {keywords[-1]}."

def make_korean_caption_from_components(components):
    """캡션 컴포넌트를 한글 문장으로 변환"""
    keywords = []

    # 컴포넌트에서 한글 키워드 추출
    for key, value in components.items():
        if isinstance(value, dict) and 'korean_keyword' in value:
            keywords.append(value['korean_keyword'])

    if not keywords:
        return ""

    if len(keywords) == 1:
        return f"이 메이크업은 {keywords[0]} 스타일입니다."
    elif len(keywords) == 2:
        return f"이 메이크업은 {keywords[0]}과 {keywords[1]}이 특징입니다."
    else:
        return f"이 메이크업은 {', '.join(keywords[:-1])}, 그리고 {keywords[-1]}이 특징입니다."

# ====== 배치 파일 처리 ======
def process_batch_files():
    """배치 파일들을 처리하여 문장 생성"""

    print("📝 배치 파일들을 문장으로 변환 중...")

    # batch_*.json 파일들 찾기
    batch_files = []
    for file_name in os.listdir(input_dir):
        if file_name.startswith("batch_") and file_name.endswith(".json"):
            batch_files.append(file_name)

    batch_files.sort()
    print(f"📁 발견된 배치 파일: {len(batch_files)}개")

    for file_name in batch_files:
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name.replace('.json', '_sentences.json'))

        try:
            with open(input_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 각 이미지 결과에 문장 추가
            for item in data:
                if "captions" in item:
                    # 캡션 타입1 문장 생성
                    if "caption_type1" in item["captions"]:
                        components = item["captions"]["caption_type1"].get("components", {})
                        item["captions"]["caption_type1"]["sentence_english"] = make_caption_from_components(components)
                        item["captions"]["caption_type1"]["sentence_korean"] = make_korean_caption_from_components(components)

                    # 캡션 타입2 문장 생성
                    if "caption_type2" in item["captions"]:
                        components = item["captions"]["caption_type2"].get("components", {})
                        item["captions"]["caption_type2"]["sentence_english"] = make_caption_from_components(components)
                        item["captions"]["caption_type2"]["sentence_korean"] = make_korean_caption_from_components(components)

            # 변환된 데이터 저장
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

            print(f"✅ {file_name} → {os.path.basename(output_path)} 변환 완료 ({len(data)}개)")

        except Exception as e:
            print(f"❌ {file_name} 처리 실패: {e}")

# ====== 통합 파일 처리 ======
def process_final_file():
    """최종 통합 파일 처리"""

    final_input_path = os.path.join(input_dir, "all_celebrity_makeup_captions.json")
    final_output_path = os.path.join(output_dir, "all_celebrity_makeup_sentences.json")

    if not os.path.exists(final_input_path):
        print("❌ 통합 파일이 존재하지 않습니다.")
        return

    print(f"\n📝 통합 파일 처리 중...")

    try:
        with open(final_input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 각 이미지 결과에 문장 추가
        for item in data:
            if "captions" in item:
                # 캡션 타입1 문장 생성
                if "caption_type1" in item["captions"]:
                    components = item["captions"]["caption_type1"].get("components", {})
                    item["captions"]["caption_type1"]["sentence_english"] = make_caption_from_components(components)
                    item["captions"]["caption_type1"]["sentence_korean"] = make_korean_caption_from_components(components)

                # 캡션 타입2 문장 생성
                if "caption_type2" in item["captions"]:
                    components = item["captions"]["caption_type2"].get("components", {})
                    item["captions"]["caption_type2"]["sentence_english"] = make_caption_from_components(components)
                    item["captions"]["caption_type2"]["sentence_korean"] = make_korean_caption_from_components(components)

        # 변환된 데이터 저장
        with open(final_output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        print(f"✅ 통합 파일 변환 완료: {final_output_path} ({len(data)}개)")

        # 통계 생성
        type1_count = sum(1 for item in data if "caption_type1" in item.get("captions", {}))
        type2_count = sum(1 for item in data if "caption_type2" in item.get("captions", {}))

        print(f"📊 캡션 타입1 문장: {type1_count}개")
        print(f"📊 캡션 타입2 문장: {type2_count}개")

        return data

    except Exception as e:
        print(f"❌ 통합 파일 처리 실패: {e}")

# ====== 결과 미리보기 ======
def show_sentence_samples(num_samples=5):
    """생성된 문장 샘플 보기"""

    final_output_path = os.path.join(output_dir, "all_celebrity_makeup_sentences.json")

    if not os.path.exists(final_output_path):
        print("❌ 문장 파일이 존재하지 않습니다.")
        return

    with open(final_output_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"\n👀 생성된 문장 샘플 ({num_samples}개):")
    print("=" * 80)

    for i, item in enumerate(data[:num_samples]):
        print(f"\n🖼️ {i+1}. {item['image_name']} ({item['celebrity_name']})")

        if "captions" in item:
            if "caption_type1" in item["captions"]:
                caption1 = item["captions"]["caption_type1"]
                print(f"📝 타입1 (한글): {caption1.get('sentence_korean', 'N/A')}")
                print(f"📝 타입1 (영어): {caption1.get('sentence_english', 'N/A')}")

            if "caption_type2" in item["captions"]:
                caption2 = item["captions"]["caption_type2"]
                print(f"📝 타입2 (한글): {caption2.get('sentence_korean', 'N/A')}")
                print(f"📝 타입2 (영어): {caption2.get('sentence_english', 'N/A')}")

        print("-" * 60)

# ====== 메인 실행 ======
def convert_captions_to_sentences():
    """메인 실행 함수"""

    print("🚀 메이크업 캡션을 문장으로 변환 시작!")

    # 1. 배치 파일들 처리
    process_batch_files()

    # 2. 통합 파일 처리
    data = process_final_file()

    # 3. 결과 미리보기
    if data:
        show_sentence_samples(5)

    print(f"\n🎉 변환 완료!")
    print(f"📂 출력 폴더: {output_dir}")

# 실행
if __name__ == "__main__":
    convert_captions_to_sentences()

### ai pipeline에 써둔 형식으로 저장하기

In [ ]:
import os
import json

# 경로 설정
input_dir = "/content/drive/MyDrive/korean_celebrity_makeup_captions"
output_dir = "/content/drive/MyDrive/korean_celebrity_makeup_formatted"
os.makedirs(output_dir, exist_ok=True)

# ====== 데이터 변환 함수 ======
def convert_to_requested_format(item):
    """기존 데이터를 요청된 형태로 변환"""

    # request 섹션
    request = {
        "이미지경로": item.get("image_path", "")
    }

    # response 섹션 초기화
    response = {
        "predicted_keywords": [],
        "caption": "",
        "prompt_en": ""
    }

    # 캡션 타입1 우선 사용 (분위기 + 메이크업 디테일 2개)
    captions = item.get("captions", {})

    if "caption_type1" in captions:
        caption_data = captions["caption_type1"]
        components = caption_data.get("components", {})

        # predicted_keywords 생성
        for comp_key, comp_value in components.items():
            if isinstance(comp_value, dict):
                keyword_entry = {
                    "keyword_kr": comp_value.get("korean_keyword", ""),
                    "keyword_en": comp_value.get("english_keyword", ""),
                    "score": round(comp_value.get("score", 0.0), 4)
                }
                response["predicted_keywords"].append(keyword_entry)

        # caption 생성 (자연스러운 문장)
        keywords_en = [kw["keyword_en"] for kw in response["predicted_keywords"]]
        if len(keywords_en) == 1:
            response["caption"] = f"A woman with {keywords_en[0]} makeup."
        elif len(keywords_en) == 2:
            response["caption"] = f"A woman with {keywords_en[0]} and {keywords_en[1]} makeup."
        elif len(keywords_en) >= 3:
            response["caption"] = f"A woman with {', '.join(keywords_en[:-1])}, and {keywords_en[-1]} makeup."

        # prompt_en 생성 (메이크업 추천용)
        if keywords_en:
            response["prompt_en"] = f"{', '.join(keywords_en)} makeup, suitable for natural beauty."

    # 캡션 타입1이 없으면 타입2 사용
    elif "caption_type2" in captions:
        caption_data = captions["caption_type2"]
        components = caption_data.get("components", {})

        # predicted_keywords 생성
        for comp_key, comp_value in components.items():
            if isinstance(comp_value, dict):
                keyword_entry = {
                    "keyword_kr": comp_value.get("korean_keyword", ""),
                    "keyword_en": comp_value.get("english_keyword", ""),
                    "score": round(comp_value.get("score", 0.0), 4)
                }
                response["predicted_keywords"].append(keyword_entry)

        # caption 생성
        keywords_en = [kw["keyword_en"] for kw in response["predicted_keywords"]]
        if len(keywords_en) == 1:
            response["caption"] = f"A woman with {keywords_en[0]} makeup."
        elif len(keywords_en) == 2:
            response["caption"] = f"A woman with {keywords_en[0]} and {keywords_en[1]} makeup."
        elif len(keywords_en) >= 3:
            response["caption"] = f"A woman with {', '.join(keywords_en[:-1])}, and {keywords_en[-1]} makeup."

        # prompt_en 생성
        if keywords_en:
            response["prompt_en"] = f"{', '.join(keywords_en)} makeup, suitable for natural beauty."

    # 최종 형태
    result = {
        "request": request,
        "response": response
    }

    return result

# ====== 배치 파일 처리 ======
def process_batch_files():
    """배치 파일들을 요청된 형태로 변환"""

    print("🔄 배치 파일들을 요청된 형태로 변환 중...")

    # batch_*.json 파일들 찾기
    batch_files = []
    for file_name in os.listdir(input_dir):
        if file_name.startswith("batch_") and file_name.endswith(".json"):
            batch_files.append(file_name)

    batch_files.sort()
    print(f"📁 발견된 배치 파일: {len(batch_files)}개")

    all_converted_data = []

    for file_name in batch_files:
        input_path = os.path.join(input_dir, file_name)

        try:
            with open(input_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 각 이미지 결과를 요청된 형태로 변환
            converted_batch = []
            for item in data:
                converted_item = convert_to_requested_format(item)
                converted_batch.append(converted_item)
                all_converted_data.append(converted_item)

            # 변환된 배치 저장
            output_path = os.path.join(output_dir, file_name.replace('.json', '_formatted.json'))
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(converted_batch, f, indent=2, ensure_ascii=False)

            print(f"✅ {file_name} → {os.path.basename(output_path)} 변환 완료 ({len(converted_batch)}개)")

        except Exception as e:
            print(f"❌ {file_name} 처리 실패: {e}")

    return all_converted_data

# ====== 통합 파일 처리 ======
def process_final_file():
    """최종 통합 파일을 요청된 형태로 변환"""

    final_input_path = os.path.join(input_dir, "all_celebrity_makeup_captions.json")
    final_output_path = os.path.join(output_dir, "all_celebrity_makeup_formatted.json")

    if not os.path.exists(final_input_path):
        print("❌ 통합 파일이 존재하지 않습니다.")
        return []

    print(f"\n🔄 통합 파일 처리 중...")

    try:
        with open(final_input_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 각 이미지 결과를 요청된 형태로 변환
        converted_data = []
        for item in data:
            converted_item = convert_to_requested_format(item)
            converted_data.append(converted_item)

        # 변환된 데이터 저장
        with open(final_output_path, "w", encoding="utf-8") as f:
            json.dump(converted_data, f, indent=2, ensure_ascii=False)

        print(f"✅ 통합 파일 변환 완료: {final_output_path} ({len(converted_data)}개)")

        # 통계 생성
        valid_count = sum(1 for item in converted_data if item["response"]["predicted_keywords"])
        avg_keywords = sum(len(item["response"]["predicted_keywords"]) for item in converted_data) / len(converted_data) if converted_data else 0

        print(f"📊 유효한 캡션: {valid_count}개")
        print(f"📊 평균 키워드 수: {avg_keywords:.1f}개")

        return converted_data

    except Exception as e:
        print(f"❌ 통합 파일 처리 실패: {e}")
        return []

# ====== 결과 미리보기 ======
def show_formatted_samples(num_samples=3):
    """변환된 결과 샘플 보기"""

    final_output_path = os.path.join(output_dir, "all_celebrity_makeup_formatted.json")

    if not os.path.exists(final_output_path):
        print("❌ 변환된 파일이 존재하지 않습니다.")
        return

    with open(final_output_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"\n👀 변환된 결과 샘플 ({num_samples}개):")
    print("=" * 100)

    for i, item in enumerate(data[:num_samples]):
        print(f"\n📝 샘플 {i+1}:")
        print(json.dumps(item, indent=2, ensure_ascii=False))
        print("-" * 80)

# ====== 단일 샘플 출력 (요청된 정확한 형태) ======
def show_single_sample():
    """요청된 정확한 형태의 단일 샘플 출력"""

    final_output_path = os.path.join(output_dir, "all_celebrity_makeup_formatted.json")

    if not os.path.exists(final_output_path):
        print("❌ 변환된 파일이 존재하지 않습니다.")
        return

    with open(final_output_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if data:
        print(f"\n📝 요청된 형태의 샘플:")
        print(json.dumps(data[0], indent=2, ensure_ascii=False))

# ====== 검증 함수 ======
def validate_format(data_sample):
    """요청된 형태와 일치하는지 검증"""

    required_structure = {
        "request": ["이미지경로"],
        "response": ["predicted_keywords", "caption", "prompt_en"]
    }

    keyword_structure = ["keyword_kr", "keyword_en", "score"]

    # 기본 구조 검증
    for section, fields in required_structure.items():
        if section not in data_sample:
            return False, f"'{section}' 섹션이 없습니다."

        for field in fields:
            if field not in data_sample[section]:
                return False, f"'{section}.{field}' 필드가 없습니다."

    # keywords 구조 검증
    keywords = data_sample["response"]["predicted_keywords"]
    if not isinstance(keywords, list):
        return False, "predicted_keywords가 리스트가 아닙니다."

    for i, keyword in enumerate(keywords):
        for field in keyword_structure:
            if field not in keyword:
                return False, f"keyword[{i}]에 '{field}' 필드가 없습니다."

    return True, "형태가 올바릅니다."

# ====== 메인 실행 ======
def convert_to_requested_format_main():
    """메인 실행 함수"""

    print("🚀 메이크업 캡션을 요청된 형태로 변환 시작!")

    # 1. 배치 파일들 처리
    all_data = process_batch_files()

    # 2. 통합 파일 처리
    final_data = process_final_file()

    # 3. 결과 미리보기
    if final_data:
        show_single_sample()

        # 4. 형태 검증
        is_valid, message = validate_format(final_data[0])
        print(f"\n✅ 형태 검증: {message}")

    print(f"\n🎉 변환 완료!")
    print(f"📂 출력 폴더: {output_dir}")
    print(f"📁 주요 파일: all_celebrity_makeup_formatted.json")

# 실행
if __name__ == "__main__":
    convert_to_requested_format_main()